In [6]:
%load_ext memory_profiler

from autograd.differential_operators import binomial_checkpoint
import autograd.numpy as np
from autograd import grad

from time import time

seq_len = 1000

param_shape = (300,)
state_shape = (300,)
x_shape = (400,)

inputs = [np.random.randn(*x_shape) for _ in range(seq_len)]
initial_state = np.random.randn(*state_shape)
parameters = np.ones(param_shape)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [7]:
def simple_rnn(param, state, x):
    assert(param.shape == param_shape)
    assert(state.shape == state_shape)
    assert(x.shape == x_shape)
    
    return param * (state / np.sum(x))

loop, unwrapped_loop = binomial_checkpoint(simple_rnn, seq_len, 100)

In [8]:
f = lambda parameters: loop(parameters, initial_state, inputs)
g = lambda parameters: np.sum(sum(f(parameters)))

start = time()
%memit g1 = (grad(g)(parameters))
end = time()

print(end - start)

f = lambda parameters: unwrapped_loop(parameters, initial_state, inputs)
g = lambda parameters: np.sum(sum(f(parameters)))

start = time()
%memit g2 = (grad(g)(parameters))
end = time()

print(end - start)

print(np.sum((g1 - g2)**2))

peak memory: 95.41 MiB, increment: 0.18 MiB
3.0937459468841553
peak memory: 103.46 MiB, increment: 8.04 MiB
4.3418869972229
2.1626198461555313e-32
